## Import

In [3]:
import glob
from math import sqrt
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
import geopandas as gpd
# import geoparquet as gpq
import math
from shapely.geometry import Point

import dask.dataframe as dd
from dask.diagnostics import ProgressBar
# import dask_geopandas as dg



from pathlib import Path
import os
import glob

import re
from tqdm import tqdm

%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Data

In [47]:
%%time
# sa
sa = gpd.read_file('C:/Users/alexd/Dropbox/Work/Repository/FFE/data/shapefile/SA2_input.shp')
sa = sa[['BldgFloo_', 'PopDensity', 'NumberBldg', 'geometry']] # drop column not used in caluclation of Ignition
sa = gpd.GeoDataFrame(sa, crs="EPSG:2193")

# pga 
pga = pd.read_parquet('/Users/alex/Dropbox/Work/Repository/OpenQuake/output/PGAxy.parquet')
pgaGeo = gpd.GeoDataFrame(pga, geometry=gpd.points_from_xy(pga.lon, pga.lat, crs="EPSG:2193"))

Wall time: 85.9 ms


In [49]:
sa

,OBJECTID,SA22018_V1,SA22018__1,LAND_AREA_,AREA_SQ_KM,Pop_Total_,SA2_ID,Shape_Leng,Shape_Area,PopDensity,BldgFloor_,BldgFloo_1,NumberBldg,geometry
0,1.0,247600,Tawa North,1.444049,1.444049,3189,1,5947.099620,1.444043e+06,2208,2341.796350,0.217560,1290,"POLYGON ((1753366.394 5441813.387, 1753328.463..."
1,2.0,249400,Karori Park,2.027442,2.027442,4566,2,7681.294158,2.027381e+06,2252,3173.889681,0.294864,1935,"POLYGON ((1744852.500 5427971.793, 1744716.795..."
2,3.0,250200,Khandallah South,0.725088,0.725088,2004,3,4517.739740,7.249921e+05,2764,2025.907871,0.188213,848,"POLYGON ((1750419.007 5431758.906, 1750362.620..."
3,4.0,250500,Onslow,0.767955,0.767955,1818,4,4741.669492,7.679913e+05,2367,1424.119168,0.132305,737,"POLYGON ((1751331.976 5432513.968, 1751386.822..."
4,5.0,251000,Wellington Botanic Gardens,0.525400,0.525400,1119,5,3491.036610,5.254374e+05,2130,940.916465,0.087414,243,"POLYGON ((1748516.704 5428769.002, 1748516.708..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,72.0,254600,Karaka Bay-Worser Bay,0.674159,0.674159,1503,72,7220.552654,6.742336e+05,2229,1417.951447,0.131732,741,"POLYGON ((1753314.360 5425868.100, 1753324.786..."
72,73.0,254700,Houghton Bay,1.287949,1.287949,1677,73,8002.213765,1.287853e+06,1302,1152.943973,0.107112,723,"POLYGON ((1749820.120 5423262.735, 1749790.522..."
73,74.0,255000,Rongotai,1.923700,1.923700,48,74,12925.117061,1.923907e+06,25,1652.852264,0.153555,133,"POLYGON ((1751454.143 5424576.656, 1751437.292..."
74,75.0,255100,Strathmore (Wellington City),1.482935,1.482935,3828,75,6541.098760,1.483021e+06,2581,2482.082395,0.230593,1579,"POLYGON ((1753062.244 5423610.494, 1753054.409..."


In [4]:
# %%time
# del pga
# data = gpd.sjoin(pgaGeo, sa, op="within")

CPU times: user 35min 36s, sys: 8min 49s, total: 44min 26s
Wall time: 1h 42min 30s


In [5]:
# %%time
# data.to_file('/Users/alex/Dropbox/Work/GNS/008_FFE/ProbaFFE/FFE/shapefile/joinSA_PGA.shp')

## Ignition calculations

In [ ]:
# for windows
# path_on_windows = PureWindowsPath(filename)

In [4]:
%%time
data = gpd.read_file('C:/Users/alexd/Dropbox/Work/GNS/008_FFE/ProbaFFE/FFE/shapefile/joinSA_PGA.shp')

Wall time: 30min 53s


In [9]:
test = data.head(20)

In [23]:
from math import *

In [24]:
math.exp(0.5)

1.6487212707001282

In [38]:
PGA = 0.5
PD = 2000 # pop density per km2
SF = 1000 # total building square footage

Pign = math.exp(-6.755 + (8.463 * PGA) + (98.4e-6 * PD) + (152.3e-6 * SF)) / (1 + math.exp(-6.755 + (8.463 * PGA) + (98.4e-6 * PD) + (152.3e-6 * SF)))
number

0.10207304854560409

In [40]:
NB = 1000 # number of building
PignBld = 1 - math.pow(1-Pign, 1/NB)
PignBld

0.00010766076397183966

In [29]:
test

,__null_das,event_id,site_id,gmv_PGA,lon,lat,index_righ,BldgFloo_1,PopDensity,NumberBldg,geometry,SA_IgnPrb,IgnProb_bl
0,0,0,0,0.050537,1.752780e+06,5.441405e+06,0,0.21756,2208,1290,POINT (1752779.830 5441405.376),inf,-inf
1,76,1,0,0.259768,1.752780e+06,5.441405e+06,0,0.21756,2208,1290,POINT (1752779.830 5441405.376),inf,-inf
2,152,2,0,0.099314,1.752780e+06,5.441405e+06,0,0.21756,2208,1290,POINT (1752779.830 5441405.376),inf,-inf
3,228,3,0,0.128333,1.752780e+06,5.441405e+06,0,0.21756,2208,1290,POINT (1752779.830 5441405.376),inf,-inf
4,304,4,0,0.017772,1.752780e+06,5.441405e+06,0,0.21756,2208,1290,POINT (1752779.830 5441405.376),inf,-inf
5,380,5,0,0.040890,1.752780e+06,5.441405e+06,0,0.21756,2208,1290,POINT (1752779.830 5441405.376),inf,-inf
6,456,6,0,0.226107,1.752780e+06,5.441405e+06,0,0.21756,2208,1290,POINT (1752779.830 5441405.376),inf,-inf
7,532,7,0,0.053637,1.752780e+06,5.441405e+06,0,0.21756,2208,1290,POINT (1752779.830 5441405.376),inf,-inf
8,608,8,0,0.031728,1.752780e+06,5.441405e+06,0,0.21756,2208,1290,POINT (1752779.830 5441405.376),inf,-inf
9,684,9,0,0.099966,1.752780e+06,5.441405e+06,0,0.21756,2208,1290,POINT (1752779.830 5441405.376),inf,-inf


In [15]:
test['SA_IgnPrb'] = 1/(np.exp(-1*(-6.755+8.463+test['gmv_PGA']+0.0000984+test['PopDensity']+0.001523+test['BldgFloo_1'])))
test['IgnProb_bl'] = 1 - np.power(1-test['SA_IgnPrb'], +1/test['NumberBldg'])

In [19]:
(-1*(-6.755+8.463+test['gmv_PGA']+0.0000984+test['PopDensity']+0.001523+test['BldgFloo_1']))

0    -2209.977718
1    -2210.186949
2    -2210.026495
3    -2210.055515
4    -2209.944954
5    -2209.968072
6    -2210.153288
7    -2209.980819
8    -2209.958909
9    -2210.027147
10   -2210.149968
11   -2210.072399
12   -2209.969594
13   -2209.961594
14   -2209.955998
15   -2209.972846
16   -2209.963446
17   -2209.974335
18   -2209.980306
19   -2209.979750
dtype: float64

In [33]:
98.4e-6


9.84e-05